In [1]:
import torchvision
print(torchvision.__version__)
%matplotlib inline

In [2]:
import os, sys
from google.colab import drive
drive.mount('/content/drive') # only run when creating notebook for the first time
sys.path.append('/content/drive/MyDrive/final_project')

In [3]:
# TODO: Fill in the Google Drive path where you uploaded assignment1
# Example: If you create a Fall2023 folder and put all the files under A1 folder, then 'Fall2023/A1'
GOOGLE_DRIVE_PATH_POST_MYDRIVE = 'final_project'
GOOGLE_DRIVE_PATH = os.path.join('/content', 'drive', 'MyDrive', GOOGLE_DRIVE_PATH_POST_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [5]:
from utils import Config
from data_loader import MultiOmicsDataset, create_dataloaders

In [6]:
# # Reload all modules every time before executing code
%load_ext autoreload
%autoreload 2

### 7. Run pytest

In [7]:
# %%bash
# pytest -v

In [8]:
import json
from pathlib import Path
from sklearn.metrics import classification_report, accuracy_score
import torch

In [9]:
from utils import Config
from data_loader import MultiOmicsDataset, create_dataloaders
from models import MultiOmicsClassifier, SingleOmicsClassifier
from trainers import BaseTrainer
from losses import MultiOmicsLoss, RNASeqLoss

In [10]:
from trainers import CallbackTrainer
from callbacks import LossRecorderCallback, TSNERecorderCallback
from callbacks import AccuracyRecorderCallback
from callbacks import AttentionLoggerCallback
from utils import plot_tsne, plot_loss

def extract_latent(outputs):
    """Example: Extract latent representation from your specific model outputs"""
    return outputs['latent_rep']  # Replace with your actual latent key
    # Or if using concatenated features:
    # return torch.cat([outputs['mirna_latent'], outputs['rna_latent']], dim=1)

In [11]:
def get_training_parameters(trainer, include_model_info=False, include_optimizer_state=False):
    """Extracts all relevant training parameters in a structured dictionary.

    Args:
        trainer: BaseTrainer instance
        include_model_info: Whether to include model architecture details
        include_optimizer_state: Whether to include optimizer state details

    Returns:
        Dictionary containing all training parameters
    """
    params = {
        "training": {
            "device": str(trainer.device),
        },
        "loss": {
            "type": type(trainer.loss_fn).__name__,
            "beta": getattr(trainer.loss_fn, 'target_beta', None),
            "use_focal": getattr(trainer.loss_fn, 'use_focal', None),
            "focal_gamma": getattr(trainer.loss_fn, 'focal_gamma', None),
            "label_smoothing": getattr(trainer.loss_fn, 'label_smoothing', None),
            "kl_epsilon": getattr(trainer.loss_fn, 'kl_epsilon', None)
        },
        "optimizer": {
            "type": type(trainer.optimizer).__name__,
            "lr": trainer.optimizer.param_groups[0]['lr'],
            "betas": trainer.optimizer.param_groups[0].get('betas', None),
            "eps": trainer.optimizer.param_groups[0].get('eps', None),
            "weight_decay": trainer.optimizer.param_groups[0].get('weight_decay', None)
        }
    }

    if include_model_info:
        params["model"] = {
            "type": type(trainer.model).__name__,
            "total_parameters": sum(p.numel() for p in trainer.model.parameters()),
            "trainable_parameters": sum(p.numel() for p in trainer.model.parameters()
                                      if p.requires_grad),
            "architecture": str(trainer.model)  # This shows the model structure
        }

    if include_optimizer_state:
        params["optimizer"]["state"] = {
            "momentum_buffer": any('momentum_buffer' in p for p in trainer.optimizer.state.values())
        }


    return params

# RNA-seq Only Models

In [12]:
config = Config.from_yaml("/content/drive/MyDrive/final_project/configs/data_rna_config.yaml")

dataset = MultiOmicsDataset(config)
dataloaders = create_dataloaders(dataset, config)

In [13]:
#Apoorva
rna_exp_dim = 13054
latent_dim = 16
num_classes = 5

# Instantiate model
singleomics_model = SingleOmicsClassifier(
    rna_exp_dim=rna_exp_dim,
    latent_dim=latent_dim,
    num_classes=num_classes
)

In [14]:
trainer = BaseTrainer(
    model=singleomics_model,
    optimizer=torch.optim.Adam(singleomics_model.parameters(), lr=1e-3),
    loss_fn=RNASeqLoss(),
    device=device
)

trainer.fit(train_loader=dataloaders['train'], val_loader=dataloaders['val'], epochs=10)

In [15]:
preds, targets = trainer.predict(dataloaders["test"])

# Compute accuracy
acc = accuracy_score(targets.numpy(), preds.numpy())
print(f"✅ Test Accuracy: {acc:.4f}")

In [16]:
print(classification_report(targets.numpy(), preds.numpy()))

params = get_training_parameters(trainer)
print(json.dumps(params, indent=4))

In [17]:
model_no = 0
loss_fn = RNASeqLoss(
    beta=0.25,               # Target KL weight
    # annealing_steps=5000,   # Total steps to anneal over
    # use_focal=True,
    # focal_gamma=1,
    # label_smoothing=0.1,
    # class_weights=None # or torch.tensor([...], device="cuda")
)

rna_exp_dim = 13054
latent_dim = 32
num_classes = 5

# Instantiate model
singleomics_model = SingleOmicsClassifier(
    rna_exp_dim=rna_exp_dim,
    latent_dim=latent_dim,
    num_classes=num_classes
)



loss_callback = LossRecorderCallback(save_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/loss_history_{0}.json".format(model_no))
tsne_callback = TSNERecorderCallback(
    val_loader=dataloaders['val'],  # Your validation DataLoader
    device=device,
    save_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/tsne_results_{0}.pkl".format(model_no)  # Matches your plot_tsne() default
)

trainer_cb = CallbackTrainer(
    model=singleomics_model,
    optimizer=torch.optim.AdamW(singleomics_model.parameters(), lr=2e-4, weight_decay=0.01),
    loss_fn=loss_fn,
    device=device,
    callbacks=[loss_callback, tsne_callback]
)

trainer_cb.fit(train_loader=dataloaders['train'], val_loader=dataloaders['val'], epochs=50)

In [18]:
preds, targets = trainer_cb.predict(dataloaders["test"])
acc = accuracy_score(targets.numpy(), preds.numpy())
print(f"✅ Test Accuracy: {acc:.4f}")

print(classification_report(targets.numpy(), preds.numpy()))
params = get_training_parameters(trainer_cb)

print(json.dumps(params, indent=4))

In [19]:
plot_tsne(tsne_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/tsne_results_{0}.pkl".format(model_no), epochs=[1, 10, 20, 30, 40, 50], figsize=(4,3))

In [20]:
plot_loss(loss_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/loss_history_{0}.json".format(model_no), figsize=(8, 4))

In [21]:
# for name, param in multiomics_model.named_parameters():
#     print(name, param.grad is not None)

In [22]:
model_no=1
loss_fn = RNASeqLoss(
    beta=0.1,               # Target KL weight
    annealing_steps=5000,   # Total steps to anneal over
    use_focal=True,
    focal_gamma=1,
    label_smoothing=0.1,
    class_weights=torch.tensor([1.0, 2.5, 1.0, 1.0, 1.5], device=device) # or torch.tensor([...], device="cuda")
)


rna_exp_dim = 13054
latent_dim = 32
num_classes = 5

# Instantiate model
singleomics_model = SingleOmicsClassifier(
    rna_exp_dim=rna_exp_dim,
    latent_dim=latent_dim,
    num_classes=num_classes
)



loss_callback = LossRecorderCallback(save_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/loss_history_{0}.json".format(model_no))
tsne_callback = TSNERecorderCallback(
    val_loader=dataloaders['val'],  # Your validation DataLoader
    device=device,
    save_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/tsne_results_{0}.pkl".format(model_no)  # Matches your plot_tsne() default
)

trainer_cb = CallbackTrainer(
    model=singleomics_model,
    optimizer=torch.optim.Adam(singleomics_model.parameters(), lr=2e-4),
    loss_fn=loss_fn,
    device=device,
    callbacks=[loss_callback, tsne_callback],
    seed=42
)

trainer_cb.fit(train_loader=dataloaders['train'], val_loader=dataloaders['val'], epochs=50)

In [23]:
preds, targets = trainer_cb.predict(dataloaders["test"])
acc = accuracy_score(targets.numpy(), preds.numpy())
print(f"✅ Test Accuracy: {acc:.4f}")

print(classification_report(targets.numpy(), preds.numpy()))
params = get_training_parameters(trainer_cb)

print(json.dumps(params, indent=4))

In [24]:
plot_tsne(tsne_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/tsne_results_{0}.pkl".format(model_no), epochs=[1, 10, 20, 30, 40, 50], cols=3, figsize=(4, 3))

In [25]:
plot_loss(loss_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/loss_history_{0}.json".format(model_no), figsize=(6, 4))

In [26]:
model_no=2
loss_fn = RNASeqLoss(
    beta=0.25,               # Target KL weight
    # annealing_steps=5000,   # Total steps to anneal over
    use_focal=True,
    focal_gamma=2.5,
    label_smoothing=0,
    class_weights=torch.tensor([1.0, 1.0, 1.0, 1.5, 2.5], device=device) # or torch.tensor([...], device="cuda")
)

rna_exp_dim = 13054
latent_dim = 32
num_classes = 5

# Instantiate model
singleomics_model = SingleOmicsClassifier(
    rna_exp_dim=rna_exp_dim,
    latent_dim=latent_dim,
    num_classes=num_classes
)




loss_callback = LossRecorderCallback(save_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/loss_history_{0}.json".format(model_no))
acc_callback = AccuracyRecorderCallback(save_path=Path(GOOGLE_DRIVE_PATH + "/logs/rna_seq/accuracy_history_{0}.json".format(model_no)))

tsne_callback = TSNERecorderCallback(
    val_loader=dataloaders['val'],  # Your validation DataLoader
    device=device,
    save_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/tsne_results_{0}.pkl".format(model_no)  # Matches your plot_tsne() default
)



trainer_cb = CallbackTrainer(
    model=singleomics_model,
    optimizer=torch.optim.AdamW(singleomics_model.parameters(), lr=2e-4, weight_decay=0.0001),
    loss_fn=loss_fn,
    device=device,
    callbacks=[loss_callback, tsne_callback, acc_callback],
    seed=42
)

trainer_cb.fit(train_loader=dataloaders['train'], val_loader=dataloaders['val'], epochs=50)

In [27]:
from utils import set_seed
set_seed(42)
preds, targets = trainer_cb.predict(dataloaders["test"])
acc = accuracy_score(targets.numpy(), preds.numpy())
print(f"✅ Test Accuracy: {acc:.4f}")

print(classification_report(targets.numpy(), preds.numpy()))
params = get_training_parameters(trainer_cb)

print(json.dumps(params, indent=4))

In [28]:
import numpy as np
from sklearn.metrics import accuracy_score
from utils import set_seed
import torch

def evaluate_multiple_seeds(trainer, test_loader, seeds=100):
    accs = []

    for seed in np.random.randint(0, 10_000, size=seeds):
        set_seed(int(seed))  # Reset seed
        preds, targets = trainer.predict(test_loader)
        acc = accuracy_score(targets.numpy(), preds.numpy())
        accs.append(acc)

    accs = np.array(accs)
    print(f"\n✅ Avg Accuracy over {seeds} seeds: {accs.mean():.4f} ± {accs.std():.4f}")
    return accs
accs = evaluate_multiple_seeds(trainer_cb, dataloaders["test"], seeds=100)

In [29]:
import matplotlib.pyplot as plt
import numpy as np

def plot_accuracy_distribution(accs, bins=10, title="Accuracy Distribution Across Seeds"):
    plt.figure(figsize=(8, 5))
    plt.hist(accs, bins=bins, color="skyblue", edgecolor="black", alpha=0.8)
    plt.axvline(np.mean(accs), color='red', linestyle='--', label=f"Mean = {np.mean(accs):.4f}")
    plt.axvline(np.median(accs), color='green', linestyle=':', label=f"Median = {np.median(accs):.4f}")

    plt.title(title)
    plt.xlabel("Accuracy")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
plot_accuracy_distribution(accs)

In [30]:
plot_loss(loss_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/loss_history_{0}.json".format(model_no), figsize=(6, 4))

In [31]:
from utils import plot_accuracy
plot_accuracy(GOOGLE_DRIVE_PATH + "/logs/rna_seq/accuracy_history_{0}.json".format(model_no), figsize=(6, 4))

In [32]:
plot_tsne(tsne_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/tsne_results_{0}.pkl".format(model_no), epochs=[1, 10, 20, 15, 30, 40, 45, 50], cols=4, figsize=(4, 3))

In [33]:
preds, targets = trainer_cb.predict(dataloaders["test"])
acc = accuracy_score(targets.numpy(), preds.numpy())
print(f"✅ Test Accuracy: {acc:.4f}")

print(classification_report(targets.numpy(), preds.numpy()))
params = get_training_parameters(trainer_cb)

print(json.dumps(params, indent=4))

In [34]:
plot_tsne(tsne_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/tsne_results_{0}.pkl".format(model_no), epochs=[1, 5, 10, 15, 20, 25, 30], cols=4, figsize=(4, 3))

In [35]:
plot_loss(loss_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/loss_history_{0}.json".format(model_no), figsize=(6, 4))

In [36]:
preds, targets, logits = trainer_cb.predict(dataloaders["test"], return_logits=True)

In [37]:
import torch.nn.functional as F
probs = F.softmax(logits, dim=1)
predictions = torch.argmax(probs, dim=1)

predictions

In [38]:
model_no = 3
loss_fn = RNASeqLoss(
    beta=0.1,               # Target KL weight
    # annealing_steps=5000,   # Total steps to anneal over
    use_focal=True,
    focal_gamma=2.5,
    label_smoothing=0,
    class_weights=torch.tensor([1.0, 1.0, 1.0, 1.5, 2.5], device=device) # or torch.tensor([...], device="cuda")
)

rna_exp_dim = 13054
latent_dim = 64
num_classes = 5

# Instantiate model
singleomics_model = SingleOmicsClassifier(
    rna_exp_dim=rna_exp_dim,
    latent_dim=latent_dim,
    num_classes=num_classes
)




loss_callback = LossRecorderCallback(save_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/loss_history_{0}.json".format(model_no))
acc_callback = AccuracyRecorderCallback(save_path=Path(GOOGLE_DRIVE_PATH + "/logs/rna_seq/accuracy_history_{0}.json".format(model_no)))
tsne_callback = TSNERecorderCallback(
    val_loader=dataloaders['val'],  # Your validation DataLoader
    device=device,
    save_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/tsne_results_{0}.pkl".format(model_no)  # Matches your plot_tsne() default
)
attn_cb = AttentionLoggerCallback(save_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/attention_weights_{0}.pkl".format(model_no), modality_names=["rna"])




trainer_cb = CallbackTrainer(
    model=singleomics_model,
    optimizer=torch.optim.AdamW(singleomics_model.parameters(), lr=5e-4, weight_decay=1e-4),
    loss_fn=loss_fn,
    device=device,
    callbacks=[loss_callback, tsne_callback, acc_callback, attn_cb],
    seed=42
)

trainer_cb.fit(train_loader=dataloaders['train'], val_loader=dataloaders['val'], epochs=60)

In [39]:
from utils import set_seed
set_seed(42)
preds, targets = trainer_cb.predict(dataloaders["test"])
acc = accuracy_score(targets.numpy(), preds.numpy())
print(f"✅ Test Accuracy: {acc:.4f}")

print(classification_report(targets.numpy(), preds.numpy()))
params = get_training_parameters(trainer_cb)

print(json.dumps(params, indent=4))

In [40]:
plot_loss(loss_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/loss_history_{0}.json".format(model_no), figsize=(6, 4))

In [41]:
plot_tsne(tsne_path=GOOGLE_DRIVE_PATH + "/logs/rna_seq/tsne_results_{0}.pkl".format(model_no), epochs=[1, 5, 10, 15, 20, 25, 30, 40,45, 50, 60], cols=4, figsize=(4, 3))

In [42]:
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

with open(GOOGLE_DRIVE_PATH + "/logs/rna_seq/attention_weights_{0}.pkl".format(model_no), "rb") as f:
    attention_log = pickle.load(f)

# Example: plot head 0 from layer 0 at epoch 50
last_epoch = attention_log[-1]
weights = last_epoch["attn"][0][0]  # layer 0, head 0
labels = ["RNA"]
sns.heatmap(weights, annot=True, fmt=".2f", xticklabels=labels, yticklabels=labels)
plt.title(f"Attention — Layer 0 Head 0 @ Epoch {last_epoch['epoch']}")
plt.show()

In [43]:
#  Inspect file contents

file_npz = GOOGLE_DRIVE_PATH + "/data/clean_data/multimodal_data_features.npz"
with np.load(file_npz, allow_pickle=True) as data:
    print("Available keys:", list(data.keys()))
    print("RNA features shape:", data["rna_features"].shape)

In [44]:
# Get gene names and class mapping

with np.load(file_npz, allow_pickle=True) as data:
    gene_names = data["rna_features"]
    class_map = data['class_map']

gene_names, class_map

In [45]:
class_map.item()

In [46]:
from utils.interpretability import compute_average_saliency_by_class, plot_saliency_radar
import pandas as pd
# 1. Your trained model and test dataloader

singleomics_model.eval()
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
singleomics_model.to(device)

# 2. Compute average saliency for RNA modality
saliency_by_class = compute_average_saliency_by_class(
    model=singleomics_model,
    dataloader=dataloaders['test'],
    modality_key='rna',           # or 'mirna', 'methyl'
    max_per_class=50,
    device=device
)

In [47]:
id_to_type = {v:k for k, v in class_map.item().items()}
df_saliency_by_class = pd.DataFrame(saliency_by_class, index=gene_names).rename(columns=id_to_type)
df_saliency_by_class

In [48]:
top5_by_class = {cls: df_saliency_by_class[cls].nlargest(5).index.tolist()
                 for cls in df_saliency_by_class.columns}

bottom5_by_class = {cls: df_saliency_by_class[cls].nsmallest(5).index.tolist()
                    for cls in df_saliency_by_class.columns}

In [49]:
top5_by_class

In [50]:
# Extract top and bottom 5 genes per class
summary_table = []

for cls in df_saliency_by_class.columns:
    top_genes = df_saliency_by_class[cls].nlargest(5).index.tolist()
    bottom_genes = df_saliency_by_class[cls].nsmallest(5).index.tolist()

    top_genes = [g.split("|")[0] for g in top_genes]
    bottom_genes = [g.split("|")[0] for g in bottom_genes]

    summary_table.append({
        "Subtype": cls,
        "Top 5 Genes": ", ".join(top_genes),
        "Bottom 5 Genes": ", ".join(bottom_genes)
    })

df_summary = pd.DataFrame(summary_table)

In [51]:
df_summary

In [52]:
# Apply styling with renamed headers

display_df = df_summary.rename(columns={
    "Top 5 Genes": "Most Contributing Genes",
    "Bottom 5 Genes": "Least Contributing Genes"
})

styled_table = (
    display_df.style
    .set_table_styles([
        # 1. Top border
        {"selector": "", "props": [("border-top", "2px solid black !important")]},
        # 2. Header bottom border
        {"selector": "thead th", "props": [("border-bottom", "1px solid black !important")]},
        # 3. Bottom border
        {"selector": "", "props": [("border-bottom", "2px solid black !important")]},
        # Cell formatting
        {"selector": "th, td", "props": [
            ("padding", "8px"),
            ("text-align", "left"),
            ("border", "none !important")
        ]}
    ])
    .hide(axis="index")
    .set_caption("Most and Least Contributory Genes for Breast Cancer Subtype Prediction")
    .set_properties(**{'border-collapse': 'collapse'})
    # Rename columns for display only
    .set_table_styles([
        {"selector": "th.col_heading.level0", "props": [("font-weight", "bold")]},
    ], overwrite=False)
)

# Rename headers in the display (without changing DataFrame)


# Display in Jupyter
display(styled_table)

In [53]:
# Check the most

plot_saliency_radar(
    class_saliency=saliency_by_class,
    top_n=10,  # or 15
    gene_names=[gene.split("|")[0] for gene in gene_names],
    id_to_type={v:k for k, v in class_map.item().items()},
    ncol=3, figsize=(10, 10)
)